### Cloning Delta tables
We can create deep and shallow clones.

To understand these, it helps to first understand how Delta tables are saved.

NOTE: May need to remove the "null" from the last birthdate column if the previous demo was skipped

In [0]:
%sql

INSERT INTO hive_metastore.default.employees
VALUES 
(11, 'Julia', '650-303-2170', 'j@loonycorn.com', 'New York', null),
(12, 'Nora', '650-801-8734', 'n@loonycorn.com', 'Bellevue', null),
(13, 'Paul', '650-234-0123', 'p@loonycorn.com', 'Queens', null);

UPDATE hive_metastore.default.employees
SET empname = "Ben Boring"
WHERE ID = 2;

DELETE FROM hive_metastore.default.employees
WHERE ID = 3;

num_affected_rows
1


In [0]:
%sql

DESCRIBE HISTORY hive_metastore.default.employees

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
17,2024-10-28T04:39:20Z,599420620358905,contact@loonycorn.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3951670535128533),1028-032811-163l393x,15,SnapshotIsolation,false,"Map(numRemovedFiles -> 4, numRemovedBytes -> 6127, p25FileSize -> 1914, numDeletionVectorsRemoved -> 1, conflictDetectionTimeMs -> 392, minFileSize -> 1914, numAddedFiles -> 1, maxFileSize -> 1914, p75FileSize -> 1914, p50FileSize -> 1914, numAddedBytes -> 1914)",null,Databricks-Runtime/15.4.x-photon-scala2.12
16,2024-10-28T04:39:18Z,599420620358905,contact@loonycorn.com,DELETE,"Map(predicate -> [""(ID#12359L = 3)""])",null,List(3951670535128533),1028-032811-163l393x,15,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 1, numAddedChangeFiles -> 0, executionTimeMs -> 1920, numDeletionVectorsUpdated -> 1, numDeletedRows -> 1, scanTimeMs -> 967, numAddedFiles -> 0, numAddedBytes -> 0, rewriteTimeMs -> 932)",null,Databricks-Runtime/15.4.x-photon-scala2.12
15,2024-10-28T04:39:14Z,599420620358905,contact@loonycorn.com,UPDATE,"Map(predicate -> [""(ID#11634L = 2)""])",null,List(3951670535128533),1028-032811-163l393x,14,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2269, numDeletionVectorsUpdated -> 0, scanTimeMs -> 1044, numAddedFiles -> 1, numUpdatedRows -> 1, numAddedBytes -> 1524, rewriteTimeMs -> 1215)",null,Databricks-Runtime/15.4.x-photon-scala2.12
14,2024-10-28T04:39:09Z,599420620358905,contact@loonycorn.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3951670535128533),1028-032811-163l393x,13,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 3, numOutputBytes -> 1481)",null,Databricks-Runtime/15.4.x-photon-scala2.12
13,2024-10-28T04:34:00Z,599420620358905,contact@loonycorn.com,WRITE,"Map(mode -> Append, statsOnLoad -> false, partitionBy -> [])",null,List(3951670535128497),1028-032811-163l393x,12,WriteSerializable,true,"Map(numFiles -> 1, numOutputRows -> 1, numOutputBytes -> 1384)",null,Databricks-Runtime/15.4.x-photon-scala2.12
12,2024-10-28T04:33:53Z,599420620358905,contact@loonycorn.com,CHANGE COLUMN,"Map(column -> {""name"":""empname"",""type"":""string"",""nullable"":false,""metadata"":{}})",null,List(3951670535128497),1028-032811-163l393x,11,WriteSerializable,true,Map(),null,Databricks-Runtime/15.4.x-photon-scala2.12
11,2024-10-28T04:33:49Z,599420620358905,contact@loonycorn.com,CHANGE COLUMN,"Map(column -> {""name"":""empname"",""type"":""string"",""nullable"":false,""metadata"":{}})",null,List(3951670535128497),1028-032811-163l393x,10,WriteSerializable,false,Map(),null,Databricks-Runtime/15.4.x-photon-scala2.12
10,2024-10-28T04:33:20Z,599420620358905,contact@loonycorn.com,OPTIMIZE,"Map(predicate -> [], auto -> true, clusterBy -> [], zOrderBy -> [], batchId -> 0)",null,List(3951670535128497),1028-032811-163l393x,9,SnapshotIsolation,false,"Map(numRemovedFiles -> 2, numRemovedBytes -> 3222, p25FileSize -> 1738, numDeletionVectorsRemoved -> 1, minFileSize -> 1738, numAddedFiles -> 1, maxFileSize -> 1738, p75FileSize -> 1738, p50FileSize -> 1738, numAddedBytes -> 1738)",null,Databricks-Runtime/15.4.x-photon-scala2.12
9,2024-10-28T04:33:17Z,599420620358905,contact@loonycorn.com,UPDATE,"Map(predicate -> [""(id#8493L = 2)""])",null,List(3951670535128497),1028-032811-163l393x,8,WriteSerializable,false,"Map(numRemovedFiles -> 0, numRemovedBytes -> 0, numCopiedRows -> 0, numDeletionVectorsAdded -> 1, numDeletionVectorsRemoved -> 0, numAddedChangeFiles -> 0, executionTimeMs -> 2273, numDeletionVectorsUpdated -> 0, scanTimeMs -> 949, numAddedFil

#### Use DB Utils to view the contents of the Delta table location
Note that not all of the files will have the current data. Some of them may have stale data which are retained to allow time travel (roll backs) to a previous version.

In [0]:
%fs ls dbfs:/user/hive/warehouse/employees

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees/_delta_log/,_delta_log/,0,1730088121000
dbfs:/user/hive/warehouse/employees/deletion_vector_1b0fd888-75b2-42bd-b2c7-dde2598d199d.bin,deletion_vector_1b0fd888-75b2-42bd-b2c7-dde2598d199d.bin,43,1730089948000
dbfs:/user/hive/warehouse/employees/deletion_vector_21aa09de-b9f5-4dc7-844f-c077a4267adb.bin,deletion_vector_21aa09de-b9f5-4dc7-844f-c077a4267adb.bin,43,1730089680000
dbfs:/user/hive/warehouse/employees/deletion_vector_2c3c21e1-b0dc-446b-afe2-fcca8ac5d393.bin,deletion_vector_2c3c21e1-b0dc-446b-afe2-fcca8ac5d393.bin,43,1730089783000
dbfs:/user/hive/warehouse/employees/deletion_vector_3edee7ce-d356-4c9b-9b73-6a9cc8dcee26.bin,deletion_vector_3edee7ce-d356-4c9b-9b73-6a9cc8dcee26.bin,45,1730090357000
dbfs:/user/hive/warehouse/employees/deletion_vector_49d4b532-fa58-49c2-891c-fc1cf9c95f09.bin,deletion_vector_49d4b532-fa58-49c2-891c-fc1cf9c95f09.bin,43,1730089995000
dbfs:/user/hive/warehouse/employees/deletion_vector_5a49e42c-7020-414c-86c9-934774cc83a8.bin,deletion_vector_5a49e42c-7020-414c-86c9-934774cc83a8.bin,43,1730090360000
dbfs:/user/hive/warehouse/employees/deletion_vector_d5ea5fc9-6928-4114-a819-d89366821a25.bin,deletion_vector_d5ea5fc9-6928-4114-a819-d89366821a25.bin,43,1730090352000
dbfs:/user/hive/warehouse/employees/part-00000-219b1894-2592-4cda-b34f-46f20b174683.c000.snappy.parquet,part-00000-219b1894-2592-4cda-b34f-46f20b174683.c000.snappy.parquet,1914,1730090358000
dbfs:/user/hive/warehouse/employees/part-00000-35f84f4c-4ce5-4895-a3a4-b19e26815865.c000.snappy.parquet,part-00000-35f84f4c-4ce5-4895-a3a4-b19e26815865.c000.snappy.parquet,1384,1730090040000


#### Create a shallow clone
Note that num_copied_files is zero. It's just the metadata that has been replicated.

In [0]:
%sql

CREATE TABLE hive_metastore.default.employees_shallow_clone 
SHALLOW CLONE hive_metastore.default.employees

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
1914,1,null,0,0,0,0


In [0]:
%sql

DESCRIBE DETAIL hive_metastore.default.employees_shallow_clone

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,fd0a4d57-30fa-437c-adc2-17071ee2818f,hive_metastore.default.employees_shallow_clone,null,dbfs:/user/hive/warehouse/employees_shallow_clone,2024-10-28T04:02:01.411Z,2024-10-28T04:39:45Z,List(),List(),1,1914,Map(delta.enableDeletionVectors -> true),3,7,"List(checkConstraints, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 1, numDeletionVectors -> 1)"


#### The shallow clone does not contain any data

In [0]:
%fs ls dbfs:/user/hive/warehouse/employees_shallow_clone

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees_shallow_clone/_delta_log/,_delta_log/,0,1730090384000


#### Shallow clones do not inherit the version history
Note the operationParameters which poin to the source

In [0]:
%sql

DESCRIBE HISTORY hive_metastore.default.employees_shallow_clone

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2024-10-28T04:39:45Z,599420620358905,contact@loonycorn.com,CLONE,"Map(source -> hive_metastore.default.employees, sourceVersion -> 17, isShallow -> true)",null,List(3951670535128533),1028-032811-163l393x,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 1914, numCopiedFiles -> 0, copiedFilesSize -> 0, sourceNumOfFiles -> 1)",null,Databricks-Runtime/15.4.x-photon-scala2.12


#### Create a deep clone
The num_copied_files is equal to the source_num_of_files. These point to the current data in the source delta table

In [0]:
%sql

CREATE TABLE hive_metastore.default.employees_deep_clone
CLONE hive_metastore.default.employees

source_table_size,source_num_of_files,num_of_synced_transactions,num_removed_files,num_copied_files,removed_files_size,copied_files_size
1914,1,null,0,1,0,1914


In [0]:
%sql

DESCRIBE DETAIL hive_metastore.default.employees_deep_clone

format,id,name,description,location,createdAt,lastModified,partitionColumns,clusteringColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,c22d8e79-7772-4f32-baac-726de303c8f1,hive_metastore.default.employees_deep_clone,null,dbfs:/user/hive/warehouse/employees_deep_clone,2024-10-28T04:02:01.411Z,2024-10-28T04:40:40Z,List(),List(),1,1914,Map(delta.enableDeletionVectors -> true),3,7,"List(checkConstraints, deletionVectors, invariants)","Map(numRowsDeletedByDeletionVectors -> 1, numDeletionVectors -> 1)"


#### The deep clone does not maintain the version history of the original
Hence, the number of .parquet files here is fewer than in the original

In [0]:
%fs ls dbfs:/user/hive/warehouse/employees_deep_clone

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees_deep_clone/__tmp_path_dir/,__tmp_path_dir/,0,1730090439000
dbfs:/user/hive/warehouse/employees_deep_clone/_delta_log/,_delta_log/,0,1730090438000
dbfs:/user/hive/warehouse/employees_deep_clone/deletion_vector_5a49e42c-7020-414c-86c9-934774cc83a8.bin,deletion_vector_5a49e42c-7020-414c-86c9-934774cc83a8.bin,43,1730090439000
dbfs:/user/hive/warehouse/employees_deep_clone/part-00000-219b1894-2592-4cda-b34f-46f20b174683.c000.snappy.parquet,part-00000-219b1894-2592-4cda-b34f-46f20b174683.c000.snappy.parquet,1914,1730090439000


In [0]:
%sql

DESCRIBE HISTORY hive_metastore.default.employees_deep_clone

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2024-10-28T04:40:40Z,599420620358905,contact@loonycorn.com,CLONE,"Map(source -> hive_metastore.default.employees, sourceVersion -> 17, isShallow -> false)",null,List(3951670535128533),1028-032811-163l393x,-1,Serializable,false,"Map(removedFilesSize -> 0, numRemovedFiles -> 0, sourceTableSize -> 1914, numCopiedFiles -> 1, copiedFilesSize -> 1914, sourceNumOfFiles -> 1)",null,Databricks-Runtime/15.4.x-photon-scala2.12


#### The contents of the two clones are now identical
Note that the phone number for Ingrid is NULL - we will update it soon

In [0]:
%sql

SELECT * FROM hive_metastore.default.employees_shallow_clone
ORDER BY id DESC

id,empname,phone,email,city,birthdate
13,Paul,650-234-0123,p@loonycorn.com,Queens,null
12,Nora,650-801-8734,n@loonycorn.com,Bellevue,null
11,Julia,650-303-2170,j@loonycorn.com,New York,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
2,Ben Boring,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees_deep_clone
ORDER BY id DESC

id,empname,phone,email,city,birthdate
13,Paul,650-234-0123,p@loonycorn.com,Queens,null
12,Nora,650-801-8734,n@loonycorn.com,Bellevue,null
11,Julia,650-303-2170,j@loonycorn.com,New York,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
2,Ben Boring,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z


#### We update the original table
The Delta table maintains a log of changes. So original file for the updated row will still be unchanged. This means the shallow copy will still point to the old data.

In [0]:
%sql

UPDATE hive_metastore.default.employees
SET email = 'paul@loonycorn.com'
WHERE id = 13

num_affected_rows
1


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees
ORDER BY id DESC

id,empname,phone,email,city,birthdate
13,Paul,650-234-0123,paul@loonycorn.com,Queens,null
12,Nora,650-801-8734,n@loonycorn.com,Bellevue,null
11,Julia,650-303-2170,j@loonycorn.com,New York,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
2,Ben Boring,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z


#### Both clones seem unaffected by the change
This makes sense for the deep copy, but the shallow one still points to the old version of the updated record.

In [0]:
%sql

SELECT * FROM hive_metastore.default.employees_shallow_clone
ORDER BY id DESC

id,empname,phone,email,city,birthdate
13,Paul,650-234-0123,p@loonycorn.com,Queens,null
12,Nora,650-801-8734,n@loonycorn.com,Bellevue,null
11,Julia,650-303-2170,j@loonycorn.com,New York,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
2,Ben Boring,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z


In [0]:
%sql

SELECT * FROM hive_metastore.default.employees_deep_clone
ORDER BY id DESC

id,empname,phone,email,city,birthdate
13,Paul,650-234-0123,p@loonycorn.com,Queens,null
12,Nora,650-801-8734,n@loonycorn.com,Bellevue,null
11,Julia,650-303-2170,j@loonycorn.com,New York,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
2,Ben Boring,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z


### VACUUM cleaning old files

Vacuuming is a process which will remove data files that are no longer in the latest state of the transaction log for the table and are older than a retention threshold (7 days by default). VACUUM will skip all directories that begin with an underscore (_), which includes the _delta_log

We will now get rid of the file storing the original value of the updated row which the shallow clone still points to.

---------------

- delta.logRetentionDuration = "interval ": controls how long the history for a table is kept. The default is 30 days.
- delta.deletedFileRetentionDuration = "interval ": controls how long ago a file must have been deleted before being a candidate for VACUUM. The default interval is seven days.


Databricks recommends regularly running VACUUM on all tables to reduce excess cloud data storage costs. The default retention threshold for vacuum is 7 days. Setting a higher threshold gives you access to a greater history for your table, but increases the number of data files stored and, as a result, incurs greater storage costs from your cloud provider.

#### Check the table properties for the delta table
This does not explicitly have a retention period set

In [0]:
%sql

SHOW TBLPROPERTIES hive_metastore.default.employees

key,value
delta.enableDeletionVectors,true
delta.feature.checkConstraints,supported
delta.feature.deletionVectors,supported
delta.feature.invariants,supported
delta.minReaderVersion,3
delta.minWriterVersion,7


#### Set a retention period of zero

It is recommended that you set a retention interval to be at least 7 days, because old snapshots and uncommitted files can still be in use by concurrent readers or writers to the table. If VACUUM cleans up active files, concurrent readers can fail or, worse, tables can be corrupted when VACUUM deletes files that have not yet been committed. You must choose an interval that is longer than the longest running concurrent transaction and the longest period that any stream can lag behind the most recent update to the table.

In [0]:
%sql

ALTER TABLE hive_metastore.default.employees
SET TBLPROPERTIES ('delta.deletedFileRetentionDuration'= 'interval 0 hours');

SHOW TBLPROPERTIES hive_metastore.default.employees;

key,value
delta.deletedFileRetentionDuration,interval 0 hours
delta.enableDeletionVectors,true
delta.feature.checkConstraints,supported
delta.feature.deletionVectors,supported
delta.feature.invariants,supported
delta.minReaderVersion,3
delta.minWriterVersion,7


Delta Lake has a safety check to prevent you from running a dangerous VACUUM command. If you are certain that there are no operations being performed on this table that take longer than the retention interval you plan to specify, you can turn off this safety check by setting the Spark configuration property spark.databricks.delta.retentionDurationCheck.enabled to false.

In [0]:
spark.conf.get("spark.databricks.delta.retentionDurationCheck.enabled")

'true'

In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", False)

##### Purge metadata-only deletes to force data rewrite

The REORG TABLE command provides the APPLY (PURGE) syntax to rewrite data to apply soft-deletes. Soft-deletes do not rewrite data or delete data files, but rather use metadata files to indicate that some data values have changed.

REORG TABLE creates a new version of the table as the operation completes. All table versions in the history prior to this transaction refer to older data files.

In [0]:
%sql

REORG TABLE hive_metastore.default.employees APPLY (PURGE)

path,metrics
dbfs:/user/hive/warehouse/employees,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 0, true, 0, 0, 1730091129048, 1730091130769, 4, 0, null, List(0, 0), 6, 6, 0, 0, null)"


#### Vacuuming the old files
This feature needs to be used with caution. According to the docs, <i>If you run VACUUM on a Delta table, you lose the ability time travel back to a version older than the specified data retention period.</i>

In [0]:
%sql

VACUUM hive_metastore.default.employees

path
dbfs:/user/hive/warehouse/employees


#### The folder has been trimmed

In fact because of our purge command, this should have only one file

In [0]:
%fs ls dbfs:/user/hive/warehouse/employees

path,name,size,modificationTime
dbfs:/user/hive/warehouse/employees/_delta_log/,_delta_log/,0,1730088121000
dbfs:/user/hive/warehouse/employees/part-00000-e667fbca-d470-413d-97a5-cde334e5f528.c000.snappy.parquet,part-00000-e667fbca-d470-413d-97a5-cde334e5f528.c000.snappy.parquet,1878,1730090520000


### The latest version of the data is still accessible

In [0]:
%sql

SELECT * FROM hive_metastore.default.employees
ORDER BY id

id,empname,phone,email,city,birthdate
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z
2,Ben Boring,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
11,Julia,650-303-2170,j@loonycorn.com,New York,null
12,Nora,650-801-8734,n@loonycorn.com,Bellevue,null
13,Paul,650-234-0123,paul@loonycorn.com,Queens,null


#### The shallow clone cannot find the file it was pointing to
But the deep clone has been untouched by the vacuuming.

In [0]:
%sql

SELECT * FROM hive_metastore.default.employees_shallow_clone
ORDER BY id

org.apache.spark.SparkException: [FAILED_READ_FILE.DBR_FILE_NOT_EXIST] Error while reading file dbfs:/user/hive/warehouse/employees/part-00000-219b1894-2592-4cda-b34f-46f20b174683.c000.snappy.parquet. [DELTA_SHALLOW_CLONE_FILE_NOT_FOUND] File dbfs:/user/hive/warehouse/employees/part-00000-219b1894-2592-4cda-b34f-46f20b174683.c000.snappy.parquet referenced in the transaction log cannot be found. This can occur when data has been manually deleted from the file system rather than using the table `DELETE` statement. This table appears to be a shallow clone, if that is the case, this error can occur when the original table from which this table was cloned has deleted a file that the clone is still using. If you want any clones to be independent of the original table, use a DEEP clone instead. SQLSTATE: KD001
	at org.apache.spark.sql.errors.QueryExecutionErrors$.fileNotExistErrorDBR(QueryExecutionErrors.scala:1069)
	at org.apache.spark.sql.errors.QueryExecutionErrors.fileNotExistErrorDBR(Que

In [0]:
%sql

SELECT * FROM hive_metastore.default.employees_deep_clone
ORDER BY id

id,empname,phone,email,city,birthdate
1,Michelle,555-345-2222,a@loonycorn.com,King of Prussia,1991-04-21T00:00:00Z
2,Ben Boring,555-231-4521,b@loonycorn.com,Bloomington,1891-04-21T00:00:00Z
4,Fatima,555-250-2386,d@loonycorn.com,Dayton,null
5,Eduardo,555-345-2699,e@loonycorn.com,Biloxi,null
6,Amy,555-001-6723,f@loonycorn.com,Spokane,null
7,Ingrid,null,g@loonycorn.com,Houston,1983-11-07T00:00:00Z
11,Julia,650-303-2170,j@loonycorn.com,New York,null
12,Nora,650-801-8734,n@loonycorn.com,Bellevue,null
13,Paul,650-234-0123,p@loonycorn.com,Queens,null


In [0]:
%sql

SHOW TABLES in hive_metastore.default

database,tableName,isTemporary
default,employees,false
default,employees_deep_clone,false
default,employees_shallow_clone,false
,_sqldf,true


#### Clean up
To start the next demo with a clean slate

In [0]:
%sql

DROP TABLE hive_metastore.default.employees_deep_clone;
DROP TABLE hive_metastore.default.employees_shallow_clone;
DROP TABLE hive_metastore.default.employees;

SHOW TABLES in hive_metastore.default;

database,tableName,isTemporary
,_sqldf,true
